In [1]:
%load_ext autoreload

In [69]:
%autoreload
import os
from sys import path
import re
import pandas as pd
path.append('../..')
import numpy as np
from scipy.stats import ttest_rel
from relation_extraction.data.summarize import *
output_path = '/scratch/geeticka/i2b2-analyze/'
def res(path): return os.path.join(output_path, path)

relation_full_form_dictionary = {'None': 'None', 'PIP': 'PIP', 'TeCP': 'TeCP',
                                 'TeRP': 'TeRP', 'TrAP': 'TrAP', 'TrCP': 'TrCP',
                                 'TrIP': 'TrIP', 'TrNAP': 'TrNAP', 'TrWP': 'TrWP'}
relation_as_short_list = ['None', 'PIP', 'TeCP', 'TeRP', 'TrAP', 'TrCP', 'TrIP', 'TrNAP', 'TrWP']

In [70]:
confusion_matrix_portion = '<<< 8-WAY EVALUATION >>>'

In [71]:
summary_cm_baseline, summary_cm_conceptblind = print_full_summary('i2b2',
    'result-baseline-i2b2.txt', 'result-concblind-i2b2.txt', 
    'Baseline','Concept-Blind', res, relation_full_form_dictionary, relation_as_short_list, confusion_matrix_portion, 2)


TTest from Baseline to Concept-Blind
Below is the metric comparsion across the two models considering individual relations, excluding 'Other'
Metric: Precision 	 statistic -0.53 	 p_value 0.6118072355651132
Metric: Recall 	 statistic -3.45 	 p_value 0.010701830797093427
Metric: F1 	 statistic -3.93 	 p_value 0.005686085116100631

TTest from Baseline to Concept-Blind
Below is the metric comparsion across the two models considering individual relations, including 'Other'
Metric: Precision 	 statistic -0.58 	 p_value 0.5794757407237194
Metric: Recall 	 statistic -3.25 	 p_value 0.011693295800340134
Metric: F1 	 statistic -3.66 	 p_value 0.006409085463347231

All the macro and micro metrics are calculated, excluding the 'Other' class
Macro - Micro for the Baseline model
Metric: Precision 	 Macro-Micro -19.56
Metric: Recall 	 Macro-Micro -30.54
Metric: F1 	 Macro-Micro -28.21

Macro - Micro for the Concept-Blind model
Metric: Precision 	 Macro-Micro -26.89
Metric: Recall 	 Macro-Micro -32.

In [72]:
summary_cm_baseline

summary,Gold Relation,Confused With(num_examples),Correct Predictions
0,None,PIP(335) TeCP(12) TeRP(235) TrAP(153) TrCP(16)...,9515.0
1,PIP,None(460) TeRP(9) TrAP(5) TrCP(1) TrIP(41),414.0
2,TeCP,None(27) PIP(3) TeRP(61) TrAP(10) TrIP(25),62.0
3,TeRP,None(209) PIP(21) TeCP(26) TrAP(1) TrCP(2) TrI...,1411.0
4,TrAP,None(140) PIP(21) TeCP(4) TeRP(1) TrCP(4) TrIP...,576.0
5,TrCP,None(64) PIP(4) TeRP(6) TrAP(9) TrIP(23) TrNAP(2),35.0
6,TrIP,None(12) PIP(4) TeRP(1) TrAP(23) TrCP(1),10.0
7,TrNAP,None(19) PIP(4) TeRP(1) TrAP(26) TrCP(2) TrIP(7),16.0
8,TrWP,None(19) PIP(2) TeRP(1) TrAP(11) TrCP(4) TrIP(9),0.0


In [74]:
summary_cm_conceptblind

summary,Gold Relation,Confused With(num_examples),Correct Predictions
0,None,PIP(310) TeCP(25) TeRP(320) TrAP(179) TrCP(38)...,9576.0
1,PIP,None(429) TrIP(3),498.0
2,TeCP,None(26) TeRP(59) TrIP(4),99.0
3,TeRP,None(184) TeCP(23) TrIP(3),1526.0
4,TrAP,None(117) TrCP(12) TrIP(15) TrNAP(4),675.0
5,TrCP,None(52) TrAP(28) TrIP(6) TrNAP(1),56.0
6,TrIP,None(16) TrAP(19) TrCP(4) TrNAP(2),10.0
7,TrNAP,None(16) TrAP(33) TrCP(4) TrIP(2),20.0
8,TrWP,None(24) TrAP(9) TrCP(9) TrIP(3) TrNAP(1),0.0
